In [1]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import matplotlib.cm as cm
import numpy as np
import astropy.io.ascii as at
import astropy.io.fits as fits
import astropy.units as u
from astropy.time import Time
from astropy.io import fits
from astropy.coordinates import SkyCoord, Distance
from astropy.table import Table,join
# from calc_ruwe import calc_ruwe
# from scipy import stats, spatial

In [2]:
cmap2 = cm.get_cmap("viridis",7)
colors = {"IC_2391": cmap2(0),
         "IC_2602": cmap2(4),
         "NGC_2547": cmap2(3),
         "NGC_2451A": cmap2(2),
         "Collinder_135": cmap2(1)}


shapes= {"IC_2391": "o",
         "IC_2602": "d",
         "NGC_2547": "v",
         "NGC_2451A": "^",
         "Collinder_135": "s"}


In [3]:
cat_dir = os.path.expanduser("~/projects/TESS_young/")

In [4]:
from analyze_cluster_output import id_solar, read_cluster_visual

In [5]:
clusters = ["IC_2391","Collinder_135","NGC_2451A","NGC_2547","IC_2602"]
dates = ["2021-06-22","2021-06-18","2021-06-21","2021-06-21","2021-07-02"]
distances = {"Collinder_135":292,"IC_2602": 149, "NGC_2547":396,
             "IC_2391":148,"NGC_2451A":195}


In [28]:
# i = 0
# cluster = clusters[i]
max_sep = 1e3 * u.AU

for i,cluster in enumerate(clusters):

    hdbscanfile = os.path.expanduser(f"~/Dropbox/EDR3/scats/{cluster}.fits")
    with fits.open(hdbscanfile) as hdu:
        hdbscan = Table(hdu[1].data)
    if hdbscan.masked == False:
        hdbscan = Table(hdbscan, masked=True, copy=False)
    hdbscan.rename_column("GAIADER3_ID","GAIAEDR3_ID")
    hpos = SkyCoord(hdbscan["GAIAEDR3_RA"],hdbscan["GAIAEDR3_DEC"],unit=u.degree)


    dat2 = read_cluster_visual(cluster,dates[i],clean_limit=0,
                               to_plot=False,return_periodcolor=False)
    dpos = SkyCoord(dat2["GAIAEDR3_RA"],dat2["GAIAEDR3_DEC"],unit=u.degree)

    bp_rp = hdbscan["GAIAEDR3_BP"] - hdbscan["GAIAEDR3_RP"]
    bp_rp2 = dat2["GAIAEDR3_BP"] - dat2["GAIAEDR3_RP"]

    solar = id_solar(bp_rp2)

    sep_dist = max_sep.to(u.pc) / (distances[cluster]*u.pc)
    max_ang_dist =  sep_dist.value * 360 / (2*np.pi) * u.degree
    print(max_ang_dist.to(u.arcsec))

    n_solar = len(np.where(solar)[0])
    ndat = len(dat2)
    cand_resolved = np.zeros(ndat,bool)
    cand_loc = np.ones(ndat,int)*-9999

    for j in np.where(solar)[0]:    
        sep = dpos[j].separation(hpos)

        hloc = hdbscan["GAIAEDR3_ID"]==dat2["GAIAEDR3_ID"][j]

        close = (sep < max_ang_dist) & (hdbscan["GAIAEDR3_ID"]!=dat2["GAIAEDR3_ID"][j])

    #     print(len(np.where(close)[0]))
    #     print(sep[close].to(u.arcsec))
        for k in np.where(close)[0]:
            pmra_diff = np.abs(hdbscan["GAIAEDR3_PMRA"][hloc]-hdbscan["GAIAEDR3_PMRA"][k])
            pmdec_diff = np.abs(hdbscan["GAIAEDR3_PMDEC"][hloc]-hdbscan["GAIAEDR3_PMDEC"][k])
            pmra_diff_sigma = np.sqrt(hdbscan["GAIAEDR3_PMRA_ERROR"][hloc]**2 + hdbscan["GAIAEDR3_PMRA_ERROR"][k]**2)
            pmdec_diff_sigma = np.sqrt(hdbscan["GAIAEDR3_PMDEC_ERROR"][hloc]**2 + hdbscan["GAIAEDR3_PMDEC_ERROR"][k]**2)
    #         print(k,pmra_diff,pmdec_diff,pmra_diff_sigma,pmdec_diff_sigma)
            if (pmra_diff < (pmra_diff_sigma*3)) & (pmdec_diff < (pmdec_diff_sigma*3)):
                cand_resolved[j] = True
                cand_loc[j] = k
                print(i,j,k,sep[k].to(u.arcsec))
                print(k,pmra_diff_sigma,pmdec_diff_sigma)
                

    print(cluster,cand_resolved[solar])
# Something is wrong with this because I'm getting a print out for an IC2391 star, but no close targets in the boolean array

224
6.756756756756757 arcsec
IC_2391 [False False False False False False False False]
364
3.4246575342465753 arcsec
Collinder_135 [False False False False False False False False False False False False
 False False False False False]
316
5.128205128205129 arcsec
NGC_2451A [False False False False False False False False False False False]
199
2.525252525252525 arcsec
NGC_2547 [False False False False False False False False False False False False
 False]
395
6.7114093959731544 arcsec
IC_2602 [False False False False False False False False False False False False
 False False]


In [19]:
cand_resolved

array([False, False, False, False, False, False, False, False])